In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as  np
import warnings
from functions import *

warnings.filterwarnings("ignore")

/Users/juanfelipe/miniforge3/envs/stock_pred/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("../Data/prices.csv", sep=";", index_col="Date")
df.head()

,BLL,MTD,NWSA,DISH,RE,XRAY,MS,ABT,BK,ARE,...,ALL,ADI,HBAN,ENPH,FBHS,BBY,CE,WHR,SEE,STE
Date,,,,,,,,,,,,,,,,,,,,,
1962-01-02 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
tickers = df.columns

In [ ]:
for s in tickers[0:50]:

    stock_price = pd.DataFrame(df[s])
    
    stock_price.dropna(inplace=True)
    
    if stock_price.shape[0] > 0:

        training_size = round(stock_price.shape[0] * 0.80)

        Ms = MinMaxScaler()
        data = Ms.fit_transform(stock_price)

        train_data = data[:training_size]
        test_data  = data[training_size:]

        train_seq, train_label = create_sequence(train_data[:,0])
        test_seq, test_label = create_sequence(test_data[:,0])

        model = Sequential()

        model.add(LSTM(units=50, return_sequences=True, input_shape = (train_seq.shape[1], 1)))
        model.add(Dropout(0.2)) 

        model.add(LSTM(units=50))
        model.add(Dropout(0.2)) 

        model.add(Dense(1))

        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])


        model.fit(train_seq, train_label, validation_data=(test_seq, test_label) ,epochs=80, verbose=1)

        predict_seq = model.predict(test_seq)

        prediction = Ms.inverse_transform(predict_seq)

        output_data = pd.concat([stock_price.iloc[training_size+50:, 0].copy(),pd.DataFrame(prediction,columns=['close_predicted'],index=stock_price.iloc[training_size+50:].index)], axis=1)

        output_data.to_csv("../Export/Predictions/One-step/" + s + ".csv", sep=";")
        model.save("../Export/Models/One-step/" + s + ".h5")

Epoch 1/80


2023-01-27 16:08:24.591240: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


157/157 [==============================] - 5s 22ms/step - loss: 4.6199e-04 - mean_absolute_error: 0.0115 - val_loss: 8.0213e-04 - val_mean_absolute_error: 0.0207
Epoch 2/80
157/157 [==============================] - 3s 20ms/step - loss: 1.1964e-04 - mean_absolute_error: 0.0067 - val_loss: 0.0010 - val_mean_absolute_error: 0.0253
Epoch 3/80
157/157 [==============================] - 3s 20ms/step - loss: 1.1490e-04 - mean_absolute_error: 0.0064 - val_loss: 0.0052 - val_mean_absolute_error: 0.0596
Epoch 4/80
157/157 [==============================] - 3s 20ms/step - loss: 9.1012e-05 - mean_absolute_error: 0.0059 - val_loss: 0.0011 - val_mean_absolute_error: 0.0249
Epoch 5/80
157/157 [==============================] - 3s 20ms/step - loss: 8.1972e-05 - mean_absolute_error: 0.0055 - val_loss: 6.3153e-04 - val_mean_absolute_error: 0.0191
Epoch 6/80
157/157 [==============================] - 3s 20ms/step - loss: 7.9450e-05 - mean_absolute_error: 0.0057 - val_loss: 9.3632e-04 - val_mean_absolute